In [905]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import RobustScaler
import pandas as pd
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_columns', None)

In [906]:
df=pd.read_parquet('../data/processed/df.parquet')
dfd=pd.read_parquet('../data/processed/dfd.parquet')


In [907]:
#option 1

dfd=dfd.drop(columns=['marketing_carat_int','cut_int','color_int', 'clarity_int'])

# #option 2
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score'])

# #option 3 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z'])

# #option 3 bis espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','raw_density'])

# #option 3 bis2 espero que sea la mejor
# dfd=dfd.drop(columns=[ 'marketing_carat_int','cut_int','color_int', 'clarity_int',
#                      'x', 'y', 'z'])

# #option 4
# dfd=dfd.drop(columns=['marketing_carat_int', 'cut_int','color_int', 'clarity_int',
#                      'cut_color_clarity_score','x', 'y', 'z','raw_density'])




In [908]:
#option 1

df=df.drop(columns=['cut','color', 'clarity','marketing_carat'])

# #option 2
# df=df.drop(columns=['cut','color', 'clarity','marketing_carat','cut_color_clarity_score'])


In [909]:

# not_standard=['cut_Fair', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good',
#        'color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
#        'color_J', 'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2',
#        'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2',
#        'marketing_carat_bad', 'marketing_carat_good',
#        'marketing_carat_regular']

# yes_standard=[x for x in dfd.columns if x not in not_standard]

# for x in yes_standard:
#     mean = dfd[x].mean()
#     std = dfd[x].std()
#     dfd[x] = (dfd[x] - mean) / (std)

dfd.describe()

,carat,depth,table,price,x,y,z,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular
count,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000,40292.000000
mean,0.796982,61.754060,57.444761,3924.691974,5.728835,5.731202,3.538109,41.125782,3.675625,0.860425,0.029783,0.090539,0.401147,0.253003,0.225529,0.124863,0.181277,0.177554,0.208826,0.154745,0.099821,0.052914,0.013874,0.032810,0.241065,0.170629,0.150129,0.228830,0.068624,0.094039,0.023801,0.125807,0.850392
std,0.474606,1.419643,2.230838,3987.680157,1.121633,1.113513,0.692453,9.411373,0.073870,1.055642,0.169989,0.286956,0.490137,0.434738,0.417935,0.330568,0.385252,0.382141,0.406475,0.361666,0.299765,0.223864,0.116968,0.178142,0.427735,0.376189,0.357202,0.420084,0.252817,0.291886,0.152431,0.331635,0.356691
min,0.200000,52.300000,43.000000,326.000000,3.770000,3.720000,2.260000,10.000000,1.708345,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,945.000000,4.710000,4.720000,2.910000,35.000000,3.628717,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.700000,61.800000,57.000000,2396.000000,5.690000,5.710000,3.520000,41.000000,3.670067,0.490000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.040000,62.500000,59.000000,5324.000000,6.540000,6.540000,4.030000,48.000000,3.713799,1.081600,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,4.500000,79.000000,95.000000,18823.000000,10.230000,10.160000,6.720000,68.000000,4.963227,20.250000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [910]:
df


,carat,depth,table,price,x,y,z,cut_int,color_int,clarity_int,cut_color_clarity_score,raw_density,marketing_carat_int,carat_sqr
0,1.21,62.4,58.0,4268,6.83,6.79,4.25,4,1,4,31,3.683471,1,1.4641
1,0.32,63.0,57.0,505,4.35,4.38,2.75,3,3,4,34,3.664419,1,0.1024
2,0.71,65.5,55.0,2686,5.62,5.53,3.65,1,4,5,35,3.755392,1,0.5041
3,0.41,63.8,56.0,738,4.68,4.72,3.00,2,7,3,39,3.712154,1,0.1681
4,1.02,60.5,59.0,4882,6.55,6.51,3.95,5,4,3,39,3.633558,2,1.0404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40287,1.34,62.7,57.0,10070,7.10,7.04,4.43,5,4,5,47,3.630959,1,1.7956
40288,2.02,57.1,60.0,12615,8.31,8.25,4.73,2,5,2,29,3.737546,2,4.0804
40289,1.01,62.7,56.0,5457,6.37,6.42,4.01,5,3,3,36,3.695335,2,1.0201
40290,0.33,61.9,54.3,456,4.45,4.47,2.76,5,1,5,38,3.606522,1,0.1089


In [911]:
dfd

,carat,depth,table,price,x,y,z,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular
0,1.21,62.4,58.0,4268,6.83,6.79,4.25,31,3.683471,1.4641,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
1,0.32,63.0,57.0,505,4.35,4.38,2.75,34,3.664419,0.1024,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.71,65.5,55.0,2686,5.62,5.53,3.65,35,3.755392,0.5041,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,0.41,63.8,56.0,738,4.68,4.72,3.00,39,3.712154,0.1681,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,1.02,60.5,59.0,4882,6.55,6.51,3.95,39,3.633558,1.0404,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40287,1.34,62.7,57.0,10070,7.10,7.04,4.43,47,3.630959,1.7956,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
40288,2.02,57.1,60.0,12615,8.31,8.25,4.73,29,3.737546,4.0804,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
40289,1.01,62.7,56.0,5457,6.37,6.42,4.01,36,3.695335,1.0201,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0
40290,0.33,61.9,54.3,456,4.45,4.47,2.76,38,3.606522,0.1089,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1


## Split

In [912]:
data=dfd

X=data.drop(columns=['price'])
Y=data['price']

# rs = RobustScaler() 
# Xrobust = rs.fit_transform(X)
# X = pd.DataFrame(Xrobust, columns= X.columns)

## Model Generator

In [913]:

estimators=[ ('lr', LinearRegression(fit_intercept=True)),
            ('svr', DecisionTreeRegressor(random_state=0))]



machine = [LinearRegression(fit_intercept=True),
             DecisionTreeRegressor(random_state=0),
           StackingRegressor(estimators=estimators,final_estimator= LinearRegression(n_jobs = -1),cv= 5 , verbose = 2 , passthrough = True)]  

# machine = [  LinearRegression(fit_intercept=True)  ]

for x in machine:
    print(f'Model in action: {x}\n')
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.85, random_state=32)
    
#     features = RFECV(x)
#     features= features.fit(X_train, Y_train)
#     features_columns=[X.columns[pos] for pos,item in enumerate(features.support_.tolist()) if item==False]
#     X_train=X_train.drop(columns=features_columns)
#     X_test=X_test.drop(columns=features_columns)
#     print(f'Delete columns: {len(X.columns)-features.n_features_}  {features_columns} ')
        
        
    model = x  
    model.fit(X_train, Y_train)
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)

    print(f' R2 Train: {r2_score(y_true=Y_train, y_pred=predictions_train)}')
    print(f' R2 Test: {r2_score(y_true=Y_test, y_pred=predictions_test)}')
    print(f' RMSE = {np.sqrt(mean_squared_error(Y_test, predictions_test))}\n\n')
    









Model in action: LinearRegression()

 R2 Train: 0.9295522383330944
 R2 Test: 0.9295391006025808
 RMSE = 1066.1381392425633


Model in action: DecisionTreeRegressor(random_state=0)

 R2 Train: 0.9999972359273196
 R2 Test: 0.965242897932131
 RMSE = 748.7919523508253


Model in action: StackingRegressor(cv=5,
                  estimators=[('lr', LinearRegression()),
                              ('svr', DecisionTreeRegressor(random_state=0))],
                  final_estimator=LinearRegression(n_jobs=-1), passthrough=True,
                  verbose=2)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


 R2 Train: 0.9939179971117443
 R2 Test: 0.9708069490900277
 RMSE = 686.2449015274364




[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.7s finished


In [914]:
dfX_test=pd.DataFrame(X_test)
dfY_test=pd.DataFrame(Y_test)
dfpredictions_test=pd.DataFrame(predictions_test)
dftest=pd.concat([dfX_test.reset_index(drop=True),dfY_test.reset_index(drop=True),dfpredictions_test.reset_index(drop=True)], axis=1) 
dftest.rename(columns={0: 'price_predict'},inplace=True)
dftest['Diff']=dftest['price']-dftest['price_predict']
print(dftest['Diff'].describe())
dftest.loc[dftest['Diff']>2000].describe()

count    6044.000000
mean        2.816336
std       686.295900
min     -6438.403787
25%      -213.978497
50%       -21.898555
75%       211.849670
max      6218.043449
Name: Diff, dtype: float64


,carat,depth,table,x,y,z,cut_color_clarity_score,raw_density,carat_sqr,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,marketing_carat_bad,marketing_carat_good,marketing_carat_regular,price,price_predict,Diff
count,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000
mean,1.705647,61.549412,57.780000,7.618941,7.595647,4.681294,39.964706,3.685291,3.089652,0.070588,0.047059,0.329412,0.341176,0.211765,0.117647,0.094118,0.188235,0.200000,0.235294,0.129412,0.035294,0.011765,0.047059,0.223529,0.235294,0.129412,0.176471,0.094118,0.082353,0.023529,0.317647,0.658824,15253.129412,12333.959717,2919.169695
std,0.427279,1.682941,2.230674,0.738656,0.713828,0.450436,11.132799,0.066993,1.357154,0.257656,0.213021,0.472789,0.476918,0.410984,0.324102,0.293725,0.393220,0.402374,0.426700,0.337647,0.185617,0.108465,0.213021,0.419083,0.426700,0.337647,0.383482,0.293725,0.276533,0.152477,0.468324,0.476918,3284.073537,3087.141004,850.633023
min,0.340000,55.800000,51.000000,4.720000,4.600000,2.600000,22.000000,3.437398,0.115600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2160.000000,-1082.002702,2000.986632
25%,1.500000,60.600000,57.000000,7.290000,7.280000,4.500000,30.000000,3.643749,2.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13731.000000,10866.474069,2325.515357
50%,1.790000,61.700000,58.000000,7.790000,7.790000,4.810000,38.000000,3.678357,3.204100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,16068.000000,13133.330452,2682.430261
75%,2.040000,62.600000,59.000000,8.190000,8.120000,5.020000,48.000000,3.712757,4.161600,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,17713.000000,14384.631732,3242.002702
max,2.520000,67.500000,63.000000,8.900000,8.870000,5.440000,65.000000,3.883505,6.350400,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18777.000000,16482.694262,6218.043449
